In [ ]:
!pip install transformers
# !pip install datasets

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
from scipy import stats

import pandas as pd
import numpy as np

import joblib
import torch

In [ ]:
import pandas as pd

In [ ]:
%cd '/content/drive/My Drive/IIITD/SEM-7/ML/ML Project/Code/Dataset'

/content/drive/My Drive/IIITD/SEM-7/ML/ML Project/Code/Dataset


In [ ]:
train_data = pd.read_csv('train_data_cleaned.csv')
valid_data = pd.read_csv('valid_data_cleaned.csv')

In [ ]:
train_data.columns

Index(['cleaned', 'label'], dtype='object')

# Load BERT

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

## Freeze encoder 
Use only weights of head layers

In [ ]:
# for param in model.base_model.parameters():
#     param.requires_grad = False

## Generate Encodings

In [ ]:
train_text = train_data.cleaned.tolist()
valid_text = valid_data.cleaned.tolist()

In [ ]:
train_encodings = tokenizer(train_text,truncation=True,padding=True)
valid_encodings = tokenizer(valid_text,truncation=True,padding=True)

In [ ]:
class OLIdDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = OLIdDataset(train_encodings,train_data.label.tolist())
valid_dataset = OLIdDataset(valid_encodings,valid_data.label.tolist())

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1m = f1_score(labels,preds,average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'macro f1': f1m
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.497199
1000,0.345624
1500,0.204399
2000,0.111913
2500,0.066390
3000,0.043191
3500,0.031344
4000,0.029585
4500,0.013365
5000,0.017259


TrainOutput(global_step=8280, training_loss=0.08522229377486279)

In [ ]:
trainer.evaluate()

{'epoch': 20.0,
 'eval_accuracy': 0.8244186046511628,
 'eval_loss': 1.4648476839065552,
 'eval_macro f1': 0.7724465067051705}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs